In [ ]:
from __future__ import division, print_function
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys import SymbolicSys
sp.init_printing()
%matplotlib inline

In [ ]:
tend=25; y0=[0, 1]; mu=1.5

In [ ]:
plt.figure(figsize=(14, 4))
odesys1 = SymbolicSys.from_callback(lambda x, y, p: [y[1], -y[0] + p[0]*y[1]*(1 - y[0]**2)], 2, 1)
xout1, yout1, info1 = odesys1.integrate('cvode', [0, tend], y0, [mu])
odesys1.plot_result(ls=('-',), c=('k', 'r'))
len(xout1), info1

In [ ]:
def solve_and_plot(sparse=0, roots=None, nderiv=0, interpolate=None, **kwargs):
    odesys2 = SymbolicSys(zip(odesys1.dep, odesys1.exprs), odesys1.indep, odesys1.params, roots=roots)
    xout2, yout2, info2 = odesys2.integrate('cvode', [0, tend], y0, [mu], sparse=sparse, nderiv=nderiv)
    info2.pop('root_indices', None)
    if interpolate is None:
        interpolate = nderiv > 0
    plt.figure(figsize=(14, 8 if interpolate else 4))
    ax1 = plt.subplot(2 if interpolate else 1, 1, 1)
    c = ('k', 'r')
    xplot, yplot = odesys2.plot_result(interpolate=interpolate, m_lim=100,
                                       ls=('-',), c=c, **kwargs)
    
    if interpolate:
        xref, yref, inforef = odesys2.integrate('cvode', xplot, y0, [mu], force_predefined=True, nsteps=5000)
        for idx, y in enumerate(yref.T):
            plt.plot(xref, y, ls='--', c=c[idx], alpha=0.35)
        
        plt.subplots_adjust(hspace=0.0005)
        ax2 = plt.subplot(2, 1, 2, sharex=ax1)
        ydiff = yplot - yref
        for idx, y in enumerate(ydiff.T):
            plt.plot(xref, y, ls='-', c=c[idx])
        plt.setp(ax1.get_xticklabels(), visible=False)
        
    return info2, xplot, yplot

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1)
info

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs, lines=0.3)
info

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs, nderiv=1)
info

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs, nderiv=2)
info

In [ ]:
d2fdx2 = tuple(odesys1.get_jac().dot(odesys1.exprs))
d2fdx2

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2)
info

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2, nderiv=1)
info

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2, nderiv=2)
info

In [ ]:
import scipy.interpolate
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2, nderiv=0, interpolate=True,
                                    interp_from_deriv=scipy.interpolate.PchipInterpolator)